In [1]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np
import tensorflow as tf 

In [3]:
TRAIN_DIR = '.\\train'
TEST_DIR = '.\\test'

In [4]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels

In [5]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [6]:
print(train)

                                        image     label
0         .\train\angry\Training_10118481.png     angry
1         .\train\angry\Training_10120469.png     angry
2         .\train\angry\Training_10131352.png     angry
3         .\train\angry\Training_10161559.png     angry
4          .\train\angry\Training_1021836.png     angry
...                                       ...       ...
28657  .\train\surprise\Training_99916297.png  surprise
28658  .\train\surprise\Training_99924420.png  surprise
28659  .\train\surprise\Training_99937001.png  surprise
28660  .\train\surprise\Training_99951755.png  surprise
28661  .\train\surprise\Training_99984132.png  surprise

[28662 rows x 2 columns]


In [7]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [8]:
print(test)
print(test['image'])

                                        image     label
0       .\test\angry\PrivateTest_10131363.png     angry
1       .\test\angry\PrivateTest_10304478.png     angry
2        .\test\angry\PrivateTest_1054527.png     angry
3       .\test\angry\PrivateTest_10590091.png     angry
4        .\test\angry\PrivateTest_1109992.png     angry
...                                       ...       ...
7166  .\test\surprise\PublicTest_98089595.png  surprise
7167  .\test\surprise\PublicTest_98567249.png  surprise
7168  .\test\surprise\PublicTest_98972870.png  surprise
7169  .\test\surprise\PublicTest_99242645.png  surprise
7170  .\test\surprise\PublicTest_99446963.png  surprise

[7171 rows x 2 columns]
0         .\test\angry\PrivateTest_10131363.png
1         .\test\angry\PrivateTest_10304478.png
2          .\test\angry\PrivateTest_1054527.png
3         .\test\angry\PrivateTest_10590091.png
4          .\test\angry\PrivateTest_1109992.png
                         ...                   
7166    .\test\

In [9]:
from tqdm.notebook import tqdm

In [10]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    

In [11]:
train_features = extract_features(train['image']) 
# print(train['image'])

  0%|          | 0/28662 [00:00<?, ?it/s]

C:\Users\Ramkrishna Ghosh\anaconda3\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


#JPEG to PNG

In [ ]:
# from PIL import Image
# import os

# # Set the input and output folder paths
# input_folder = "C:\\Users\\Ramkrishna Ghosh\\Desktop\\New folder2"  # Change this to the path of your folder containing JPG images
# output_folder = "C:\\Users\\Ramkrishna Ghosh\\Desktop\\New folder (3)"  # Change this to the path where you want to save the PNG images

# # Create the output folder if it doesn't exist
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Get a list of all files in the input folder
# file_list = os.listdir(input_folder)

# # Loop through the files and convert JPG to PNG
# for filename in file_list:
#     if filename.endswith('.jpeg'):
#         # Open the JPG image
#         with Image.open(os.path.join(input_folder, filename)) as img:
#             # Save the image as PNG in the output folder
#             output_filename = os.path.splitext(filename)[0] + '.png'
#             img.save(os.path.join(output_folder, output_filename), 'PNG')

# print("Conversion completed.")


In [12]:
test_features = extract_features(test['image'])

  0%|          | 0/7171 [00:00<?, ?it/s]

In [13]:
x_train = train_features/255.0
x_test = test_features/255.0

In [14]:
from sklearn.preprocessing import LabelEncoder

In [15]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [16]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [17]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [18]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))


In [19]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [20]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 100, validation_data = (x_test,y_test)) 

Epoch 1/100
224/224 [==============================] - 638s 3s/step - loss: 1.8235 - accuracy: 0.2476 - val_loss: 1.8183 - val_accuracy: 0.2472
Epoch 2/100
224/224 [==============================] - 657s 3s/step - loss: 1.8038 - accuracy: 0.2525 - val_loss: 1.7888 - val_accuracy: 0.2514
Epoch 3/100
224/224 [==============================] - 726s 3s/step - loss: 1.7302 - accuracy: 0.2890 - val_loss: 1.6504 - val_accuracy: 0.3288
Epoch 4/100
224/224 [==============================] - 525s 2s/step - loss: 1.6239 - accuracy: 0.3536 - val_loss: 1.4644 - val_accuracy: 0.4281
Epoch 5/100
224/224 [==============================] - 588s 3s/step - loss: 1.5116 - accuracy: 0.4061 - val_loss: 1.3890 - val_accuracy: 0.4606
Epoch 6/100
224/224 [==============================] - 600s 3s/step - loss: 1.4530 - accuracy: 0.4399 - val_loss: 1.3415 - val_accuracy: 0.4860
Epoch 7/100
224/224 [==============================] - 601s 3s/step - loss: 1.4091 - accuracy: 0.4569 - val_loss: 1.2918 - val_accuracy:

224/224 [==============================] - 635s 3s/step - loss: 0.9422 - accuracy: 0.6422 - val_loss: 1.0226 - val_accuracy: 0.6199
Epoch 58/100
224/224 [==============================] - 877s 4s/step - loss: 0.9319 - accuracy: 0.6500 - val_loss: 1.0216 - val_accuracy: 0.6130
Epoch 59/100
224/224 [==============================] - 642s 3s/step - loss: 0.9365 - accuracy: 0.6479 - val_loss: 1.0278 - val_accuracy: 0.6143
Epoch 60/100
224/224 [==============================] - 923s 4s/step - loss: 0.9317 - accuracy: 0.6515 - val_loss: 1.0190 - val_accuracy: 0.6201
Epoch 61/100
224/224 [==============================] - 997s 4s/step - loss: 0.9254 - accuracy: 0.6527 - val_loss: 1.0190 - val_accuracy: 0.6187
Epoch 62/100
224/224 [==============================] - 761s 3s/step - loss: 0.9187 - accuracy: 0.6568 - val_loss: 1.0278 - val_accuracy: 0.6157
Epoch 63/100
224/224 [==============================] - 676s 3s/step - loss: 0.9108 - accuracy: 0.6589 - val_loss: 1.0150 - val_accuracy: 0.626

In [21]:
model_json = model.to_json()
with open(".\\emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save(".\\emotiondetector.h5")

In [22]:
from keras.models import model_from_json

In [23]:
json_file = open(".\\emotiondetector.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights(".\\emotiondetector.h5")

In [24]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [25]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [26]:
image =".\\test\\surprise\\PrivateTest_9171383.png"
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of sad


C:\Users\Ramkrishna Ghosh\anaconda3\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


1/1 [==============================] - 1s 553ms/step
model prediction is  surprise
